### Comparing different models on Tensorboard

#### Importing packages

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
                                            
torch.set_printoptions(linewidth=120)    # Display options for output
torch.set_grad_enabled(True)             # Already by default

from torch.utils.tensorboard import SummaryWriter

#### Building our Network

In [2]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Two convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        # Three fully-connected layers
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120) 
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self,t):
        
        # Forward Propagation
        
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t,kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t,kernel_size=2, stride=2)
        
        t = t.reshape(-1,12*4*4)
        t = F.relu(self.fc1(t))
        
        t = F.relu(self.fc2(t))
        
        t = self.out(t)
        
        return t

#### Dataset

In [3]:
train_set = torchvision.datasets.FashionMNIST(
    root='.data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

#### To find number of correct predictions

In [4]:
def no_of_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

#### Loading data in batches and training and visualizing on Tensorboard

In [5]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.add_image('images', grid)
tb.add_graph(network, images)

for epoch in range(5):
    
    total_loss = 0
    total_correct = 0
    
    for batch in train_loader: # Get Batch
        
        images, labels = batch
        
        preds = network(images)
        loss = F.cross_entropy(preds,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_correct += no_of_correct(preds,labels)
        
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)
    
    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)
    
    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )
    
tb.close()

KeyboardInterrupt: 

#### Accuracy of Model

In [11]:
accuracy = total_correct/len(train_set)
print('Accuracy of the model = ', accuracy*100, '%')

Accuracy of the model =  87.58666666666667 %


In [17]:
for name,weight in network.named_parameters():
    print(f'{name}.grad   ',weight.grad.shape)

conv1.weight.grad    torch.Size([6, 1, 5, 5])
conv1.bias.grad    torch.Size([6])
conv2.weight.grad    torch.Size([12, 6, 5, 5])
conv2.bias.grad    torch.Size([12])
fc1.weight.grad    torch.Size([120, 192])
fc1.bias.grad    torch.Size([120])
fc2.weight.grad    torch.Size([60, 120])
fc2.bias.grad    torch.Size([60])
out.weight.grad    torch.Size([10, 60])
out.bias.grad    torch.Size([10])
